In [1]:
import numpy as np
import random
import os
from PD.PlasmaDataset import PlasmaDataset
from PML.PlasmaModel import PlasmaModel

SyntaxError: '(' was never closed (PlasmaDataset.py, line 36)

In [ ]:
DATA_SPLIT = [0.5, 0.3, 0.2] # train/test/val splits
DATA_FRAC = 1
ORG_DATA_DIR = "./jtext_org"
HDF5_DATA_DIR = "./jtext_data/low_freq"
SETS = {}

In [ ]:
def makeDataset(dataset:"PlasmaDataset", split:list, features:list, frac:float=1, preview=False):
    dataset.initialize()
    dataset.sourceFiles(data_split = split, data_frac = frac)
    dataset.sourceData(features)
    dataset.calcStats()
    dataset.normalize()
    dataset.saveCSV(['train', 'test', 'val', 'stats'])
    if preview:
        dataset.preview()

In [ ]:
FEATS = ['ip', 'ip_error', 'dx', 'dy', 'AXUV_CE_66', 'AXUV_CE_70', 'AXUV_CE_74', 'AXUV_CE_78']

JTEXT_LOW = PlasmaDataset(org_directory = ORG_DATA_DIR, h5_source = HDF5_DATA_DIR)
makeDataset(JTEXT_LOW, split=DATA_SPLIT, frac=DATA_FRAC, features=None)